In [1]:
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_eje = pd.read_csv(r'C:\Users\LENOVO\Downloads\BASE_EJE.txt', sep=r'¡', engine='python')
base_eje.head()

## Eliminación de variables: 
base_eje2 = base_eje[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [3]:
# Vista previa
base_eje2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7245,INTERNET,16954822,15/11/2019,CALDAS,1,98049,35,8908038151,NUEVO,...,5,95,"99413,67",FISICO,Micro,EJE CAFETERO,-1.0,CALDAS,186,FISICO
1,7245,INTERNET,16663766,20/06/2019,CALDAS,1,97079,35,9007122694,NUEVO EXISTENTE,...,11,100,"93354,55",FISICO,Micro,EJE CAFETERO,-1.0,CALDAS,334,FISICO
2,7245,INTERNET,16648927,13/06/2019,CALDAS,1,97081,35,30322483,NUEVO EXISTENTE,...,11,98,"95343,64",DIGITAL,Micro,EJE CAFETERO,NaN,CALDAS,5005,DIGITAL
3,7278,EQUIPOS,16635346,05/06/2019,RISARALDA,1,192341,35,9005835505,NUEVO EXISTENTE,...,11,95,"187343,82",FISICO,Medium,EJE CAFETERO,-1.0,RISARALDA,480,NaN
4,7278,EQUIPOS,16635347,05/06/2019,RISARALDA,1,128394,35,8160063596,NUEVO EXISTENTE,...,11,95,124897,HIBRIDO,Medium,EJE CAFETERO,-1.0,TOLIMA,5816,HIBRIDO


In [4]:
## Eliminacion de clientes con datos NA
base_eje3 = base_eje2.dropna()

In [5]:
## Group by por nit para calificacion
b4 = base_eje3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_eje4 = pd.merge(base_eje3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_eje4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_eje4['y'] = pd.cut(x=base_eje4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [6]:
dataTypeSeries = base_eje4.dtypes
dataTypeSeries

cat = base_eje4.select_dtypes(include=['object','category'])
cuanti = base_eje4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [7]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.297104,-0.116290,0.316547,-0.215334,-0.207392,-0.217300,-0.007889,0.136682,-0.023667
SESUSUSC,0.297104,1.000000,0.112046,0.889573,0.050862,0.092055,0.100839,-0.226444,0.193281,0.182035
SESUCUSA,-0.116290,0.112046,1.000000,0.116712,0.418760,0.531472,0.434387,-0.134169,0.046385,0.159712
SESUCICL,0.316547,0.889573,0.116712,1.000000,0.122954,0.158772,0.140981,-0.269583,0.458343,0.262722
SUMA,-0.215334,0.050862,0.418760,0.122954,1.000000,0.980059,0.945991,-0.148240,0.131169,0.501285
CANTIDAD,-0.207392,0.092055,0.531472,0.158772,0.980059,1.000000,0.928593,-0.160542,0.141942,0.475050
CALIFICACION_VECTOR_SERVICIO_x,-0.217300,0.100839,0.434387,0.140981,0.945991,0.928593,1.000000,-0.152632,0.113143,0.525479
ESTRATO_AGRUPADO,-0.007889,-0.226444,-0.134169,-0.269583,-0.148240,-0.160542,-0.152632,1.000000,-0.220132,-0.290463
ANTIGUEDAD_DIAS,0.136682,0.193281,0.046385,0.458343,0.131169,0.141942,0.113143,-0.220132,1.000000,0.215313
CALIFICACION_VECTOR_SERVICIO_y,-0.023667,0.182035,0.159712,0.262722,0.501285,0.475050,0.525479,-0.290463,0.215313,1.000000


In [8]:
base_eje5=cuanti.join(cat)
print(base_eje5.dtypes)

#base_eje5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')

#base_eje5 = pd.read_csv(r'C:\Users\LENOVO\Documents\base_eje_process.csv', sep=',', engine='python')

X = base_eje5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        'PROM_SUSC', 'CANAL_ENVIO',
        # 'SUMA',
        #'CANTIDAD',
        #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
        'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_eje5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [9]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7245,10,0,1,27504,35,1,143430,1,4,-1.0,186,1
1,7245,10,0,1,27385,35,2,137322,1,4,-1.0,334,1
2,7278,8,5,1,3020,35,2,17491,2,3,-1.0,5816,2
3,7278,8,5,1,3020,35,2,17491,2,3,-1.0,5816,2
4,7245,10,0,1,25444,35,2,95201,1,4,-1.0,354,1


In [10]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.951204  0.972380  0.961675   96306.0   98450.0  0.915261
1             0.816123  0.804491  0.810265    7437.0    7331.0  0.955107
2             0.628104  0.482027  0.545455    8763.0    6725.0  0.843876
avg / total   0.917108  0.923089  0.919247  112506.0  112506.0  0.973484
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.935962  0.983874  0.959320   96306.0  101236.0  0.893207
1             0.728557  0.686433  0.706868    7437.0    7007.0  0.964835
2             0.613887  0.298642  0.401812    8763.0    4263.0  0.803470
avg / total   0.897165  0.910840  0.899208  112506.0  112506.0  0.973773
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.882024  0.998411  0.936616 

In [11]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.48787
Variable: 1, Score: 0.00000
Variable: 2, Score: 0.01835
Variable: 3, Score: 0.06255
Variable: 4, Score: 0.28340
Variable: 5, Score: 0.03578
Variable: 6, Score: 0.00000
Variable: 7, Score: 0.02142
Variable: 8, Score: 0.01665
Variable: 9, Score: 0.00938
Variable: 10, Score: 0.04367
Variable: 11, Score: 0.00892
Variable: 12, Score: 0.01200


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [12]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.946640  0.986460  0.966140   96306.0  100357.0  0.909030
1             0.826612  0.810273  0.818361    7437.0    7290.0  0.943083
2             0.777115  0.430903  0.554397    8763.0    4859.0  0.836908
avg / total   0.925502  0.931541  0.924301  112506.0  112506.0  0.967186
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.947862  0.987083  0.967075   96306.0  100291.0  0.918043
1             0.814182  0.813635  0.813908    7437.0    7432.0  0.960518
2             0.783818  0.427822  0.553521    8763.0    4783.0  0.847967
avg / total   0.926248  0.932057  0.924739  112506.0  112506.0  0.975652
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.949515  0.982909  0.965923   96

In [14]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

summary_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
for train_index, test_index in kf.split(X_fin, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model_final = KNeighborsClassifier(5).fit(X_train, y_train) 
    summary_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model_final.predict(X_test),
                                      y_score = model_final.predict_proba(X_test)))
 
# Mostrar resumen de metricas    
print(summary_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.906518  0.973615  0.938869   96304.0  103432.0  0.789020
1             0.604439  0.539704  0.570240    7367.0    6578.0  0.838823
2             0.368189  0.104018  0.162210    8835.0    2496.0  0.634123
avg / total   0.844463  0.876913  0.853741  112506.0  112506.0  0.938752,              precision    recall  f1-score   support      pred       AUC
0             0.902444  0.968724  0.934410   96304.0  103377.0  0.769023
1             0.586788  0.491923  0.535184    7367.0    6176.0  0.829746
2             0.332882  0.111262  0.166780    8835.0    2953.0  0.617663
avg / total   0.837048  0.870167  0.847987  112506.0  112506.0  0.933868,              precision    recall  f1-score   support      pred       AUC
0             0.899549  0.970604  0.933726   96304.0  103911.0  0.747535
1             0.580800  0.477193  0.523923    7366.0    6052.0  0.824610
2             0.327959  0.094387  0.146586    88

In [18]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, model_final.predict(X_test))

In [19]:
confusion

array([[93434,  1532,  1338],
       [ 2981,  3872,   513],
       [ 5493,  2334,  1009]], dtype=int64)